# pre-pocessing

In [1]:
import pandas as pd
from tweetNormalizer import normalizeTweet
import re

# load train and validation data
train_dataset = pd.read_json("WNUT2015_dataset/train_data.json", orient="records")
val_dataset = pd.read_json("WNUT2015_dataset/test_truth.json", orient="records")

train_dataset = train_dataset[:100]
val_dataset = val_dataset[:5]


make_sentence = lambda x : normalizeTweet(" ".join(x))

train_dataset['input_sentence'] = train_dataset['input'].apply(make_sentence)
train_dataset['output_sentence'] = train_dataset['output'].apply(make_sentence)
val_dataset['input_sentence'] = val_dataset['input'].apply(make_sentence)
val_dataset['output_sentence'] = val_dataset['output'].apply(make_sentence)

In [2]:
import torch
from transformers import BertweetTokenizer, AutoTokenizer
from tweetNormalizer import normalizeTweet

#line = "SC has first two presumptive cases of coronavirus, DHEC confirms https://postandcourier.com/health/covid19/sc-has-first-two-presumptive-cases-of-coronavirus-dhec-confirms/article_bddfe4ae-5fd3-11ea-9ce4-5f495366cee6.html?utm_medium=social&utm_source=twitter&utm_campaign=user-share… via @postandcourier"
line = "RT @EXOffical : [ HQ ] 140526 Xiumin , Luhan @ MBC Idol Futsal Championship ( cr . shade of the bloom ) http://t.co/ToBKl76SzP"


tokenizer = BertweetTokenizer.from_pretrained("vinai/bertweet-base")
input_ids = torch.tensor([tokenizer.encode(normalizeTweet(line))])
print(' '.join(tokenizer.convert_ids_to_tokens(input_ids[0], skip_special_tokens=False)))

# tokenizer = BertweetTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)
# input_ids = torch.tensor([tokenizer.encode(line)])
# print(' '.join(tokenizer.convert_ids_to_tokens(input_ids[0], skip_special_tokens=True)))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


<s> RT @USER : [ HQ ] 14@@ 05@@ 26 Xi@@ umin , Luhan @USER MBC Idol Fut@@ sal Championship ( cr . shade of the bloom ) HTTPURL </s>


In [49]:
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base", normalization=True)
special_tokens_dict = {'additional_special_tokens': ['@USER','HTTPURL']}
print(tokenizer.vocab_size)
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(num_added_toks, tokenizer.vocab_size)
tokenizer.special_tokens_map

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


64000
0 64000


{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>',
 'additional_special_tokens': "['@USER', 'HTTPURL']"}

In [50]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
special_tokens_dict = {'additional_special_tokens': ['@USER','HTTPURL']}
print(tokenizer.vocab_size)
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(num_added_toks, tokenizer.vocab_size)
tokenizer.special_tokens_map

30522
2 30522


{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]',
 'additional_special_tokens': "['@USER', 'HTTPURL']"}

In [51]:
tokenizer.additional_special_tokens

['@USER', 'HTTPURL']

In [71]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
special_tokens_dict = {'additional_special_tokens': ['@USER','HTTPURL']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(tokenizer.vocab_size)
tokenizer.special_tokens_map

50257


{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'additional_special_tokens': "['@USER', 'HTTPURL']"}

In [72]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
special_tokens_dict = {'additional_special_tokens': ['@USER','HTTPURL']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(tokenizer.vocab_size)
tokenizer.special_tokens_map

50265


{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'sep_token': '</s>',
 'pad_token': '<pad>',
 'cls_token': '<s>',
 'mask_token': '<mask>',
 'additional_special_tokens': "['@USER', 'HTTPURL']"}

In [64]:
line = "SC has first two presumptive cases of coronavirus, DHEC confirms https://postandcourier.com/health/covid19/sc-has-first-two-presumptive-cases-of-coronavirus-dhec-confirms/article_bddfe4ae-5fd3-11ea-9ce4-5f495366cee6.html?utm_medium=social&utm_source=twitter&utm_campaign=user-share… via @postandcourier"

ENCODER = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(ENCODER)

if ENCODER=="bert-base-uncased":
    # CLS token will work as BOS token, SEP token will work as EOS token
    tokenizer.bos_token = tokenizer.cls_token
    tokenizer.eos_token = tokenizer.sep_token

special_tokens_dict = {'additional_special_tokens': ['@USER','HTTPURL']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

input_ids = tokenizer.encode(normalizeTweet(line))

for rm_id in [tokenizer.bos_token_id, tokenizer.eos_token_id, tokenizer.pad_token_id]:
    if rm_id in input_ids:
        input_ids.remove(rm_id)

input_ids = torch.tensor([input_ids])

input_sent = tokenizer.decode(input_ids[0], skip_special_tokens=False)
input_sent

'sc has first two presumptive cases of coronavirus, dhec confirms HTTPURL... via @USER'

In [74]:
# set EncoderDecoderModel
model = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer

# train token classification for whether it need to be changed

In [1]:
from transformers import BertTokenizer, BertForTokenClassification
import torch
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased', return_dict=True)
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
labels = torch.tensor([1] * inputs["input_ids"].size(1)).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
# loss = outputs.loss
# logits = outputs.logits

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [2]:
labels

tensor([[1, 1, 1, 1, 1, 1, 1, 1]])

In [3]:
outputs

TokenClassifierOutput(loss=tensor(0.8729, grad_fn=<NllLossBackward>), logits=tensor([[[ 0.0837, -0.0871],
         [ 0.0103,  0.0035],
         [ 0.3324, -0.1649],
         [ 0.1886,  0.1879],
         [ 0.5739, -0.0223],
         [ 0.2863, -0.1605],
         [ 0.5136, -0.2981],
         [-0.2135, -0.1934]]], grad_fn=<AddBackward0>), hidden_states=None, attentions=None)

In [5]:
import logging
import os
import sys
from dataclasses import dataclass, field
from importlib import import_module
from typing import Dict, List, Optional, Tuple

import numpy as np
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch import nn

from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from utils_ner import Split, TokenClassificationDataset, TokenClassificationTask


logger = logging.getLogger(__name__)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    task_type: Optional[str] = field(
        default="NER", metadata={"help": "Task type to fine tune in training (e.g. NER, POS, etc)"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    data_dir: str = field(
        metadata={"help": "The input data dir. Should contain the .txt files for a CoNLL-2003-formatted task."}
    )
    labels: Optional[str] = field(
        default=None,
        metadata={"help": "Path to a file containing all labels. If not specified, CoNLL-2003 labels are used."},
    )
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    model_args = ModelArguments(model_name_or_path='bert-base-multilingual-cased', 
                                config_name=None,
                                task_type='NER', 
                                tokenizer_name=None,
                                use_fast=False,
                                cache_dir=None)
    data_args = DataTrainingArguments(data_dir='.', 
                                      labels='./labels.txt', 
                                      max_seq_length=128,
                                      overwrite_cache=False)
    training_args = TrainingArguments(output_dir='germeval-model',
                                      overwrite_output_dir=False, 
                                      do_train=True,
                                      do_eval=True, 
                                      do_predict=True,
                                      evaluate_during_training=False, 
                                      prediction_loss_only=False,
                                      per_device_train_batch_size=8, 
                                      per_device_eval_batch_size=8, 
                                      per_gpu_train_batch_size=32, 
                                      per_gpu_eval_batch_size=None, 
                                      gradient_accumulation_steps=1, 
                                      predict_from_generate=False, 
                                      learning_rate=5e-05, 
                                      weight_decay=0.0,
                                      adam_beta1=0.9,
                                      adam_beta2=0.999,
                                      adam_epsilon=1e-08,
                                      max_grad_norm=1.0,
                                      num_train_epochs=3.0, 
                                      max_steps=-1,
                                      warmup_steps=0,
                                      logging_dir='runs/Oct07_14-28-16_node002',
                                      logging_first_step=False, 
                                      logging_steps=500,
                                      save_steps=750, 
                                      save_total_limit=None,
                                      no_cuda=False,
                                      seed=1, 
                                      fp16=False, 
                                      fp16_opt_level='O1', 
                                      local_rank=-1, 
                                      tpu_num_cores=None,
                                      tpu_metrics_debug=False,
                                      debug=False,
                                      dataloader_drop_last=False, 
                                      eval_steps=1000,
                                      past_index=-1,
                                      run_name=None, 
                                      disable_tqdm=False,
                                      remove_unused_columns=True,
                                      label_names=None)


    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    module = import_module("tasks")
    try:
        token_classification_task_clazz = getattr(module, model_args.task_type)
        token_classification_task: TokenClassificationTask = token_classification_task_clazz()
    except AttributeError:
        raise ValueError(
            f"Task {model_args.task_type} needs to be defined as a TokenClassificationTask subclass in {module}. "
            f"Available tasks classes are: {TokenClassificationTask.__subclasses__()}"
        )

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    # Prepare CONLL-2003 task
    labels = token_classification_task.get_labels(data_args.labels)
    label_map: Dict[int, str] = {i: label for i, label in enumerate(labels)}
    num_labels = len(labels)

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.

    config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        num_labels=num_labels,
        id2label=label_map,
        label2id={label: i for i, label in enumerate(labels)},
        cache_dir=model_args.cache_dir,
    )
    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
        use_fast=model_args.use_fast,
    )
    model = AutoModelForTokenClassification.from_pretrained(
        model_args.model_name_or_path,
        from_tf=bool(".ckpt" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
    )

    # Get datasets
    train_dataset = (
        TokenClassificationDataset(
            token_classification_task=token_classification_task,
            data_dir=data_args.data_dir,
            tokenizer=tokenizer,
            labels=labels,
            model_type=config.model_type,
            max_seq_length=data_args.max_seq_length,
            overwrite_cache=data_args.overwrite_cache,
            mode=Split.train,
        )
        if training_args.do_train
        else None
    )
    eval_dataset = (
        TokenClassificationDataset(
            token_classification_task=token_classification_task,
            data_dir=data_args.data_dir,
            tokenizer=tokenizer,
            labels=labels,
            model_type=config.model_type,
            max_seq_length=data_args.max_seq_length,
            overwrite_cache=data_args.overwrite_cache,
            mode=Split.dev,
        )
        if training_args.do_eval
        else None
    )

    def align_predictions(predictions: np.ndarray, label_ids: np.ndarray) -> Tuple[List[int], List[int]]:
        preds = np.argmax(predictions, axis=2)

        batch_size, seq_len = preds.shape

        out_label_list = [[] for _ in range(batch_size)]
        preds_list = [[] for _ in range(batch_size)]

        for i in range(batch_size):
            for j in range(seq_len):
                if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
                    out_label_list[i].append(label_map[label_ids[i][j]])
                    preds_list[i].append(label_map[preds[i][j]])

        return preds_list, out_label_list

    def compute_metrics(p: EvalPrediction) -> Dict:
        preds_list, out_label_list = align_predictions(p.predictions, p.label_ids)
        return {
            "accuracy_score": accuracy_score(out_label_list, preds_list),
            "precision": precision_score(out_label_list, preds_list),
            "recall": recall_score(out_label_list, preds_list),
            "f1": f1_score(out_label_list, preds_list),
        }

    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
    )

    # Training
    if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)

    # Evaluation
    results = {}
    if training_args.do_eval:
        logger.info("*** Evaluate ***")

        result = trainer.evaluate()

        output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
        if trainer.is_world_master():
            with open(output_eval_file, "w") as writer:
                logger.info("***** Eval results *****")
                for key, value in result.items():
                    logger.info("  %s = %s", key, value)
                    writer.write("%s = %s\n" % (key, value))

            results.update(result)

    # Predict
    if training_args.do_predict:
        test_dataset = TokenClassificationDataset(
            token_classification_task=token_classification_task,
            data_dir=data_args.data_dir,
            tokenizer=tokenizer,
            labels=labels,
            model_type=config.model_type,
            max_seq_length=data_args.max_seq_length,
            overwrite_cache=data_args.overwrite_cache,
            mode=Split.test,
        )

        predictions, label_ids, metrics = trainer.predict(test_dataset)
        preds_list, _ = align_predictions(predictions, label_ids)

        output_test_results_file = os.path.join(training_args.output_dir, "test_results.txt")
        if trainer.is_world_master():
            with open(output_test_results_file, "w") as writer:
                for key, value in metrics.items():
                    logger.info("  %s = %s", key, value)
                    writer.write("%s = %s\n" % (key, value))

        # Save predictions
        output_test_predictions_file = os.path.join(training_args.output_dir, "test_predictions.txt")
        if trainer.is_world_master():
            with open(output_test_predictions_file, "w") as writer:
                with open(os.path.join(data_args.data_dir, "test.txt"), "r") as f:
                    token_classification_task.write_predictions_to_file(writer, f, preds_list)

    return results

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'utils_ner'

# char-based model --> too slow

In [3]:
from transformers import EncoderDecoderModel, Trainer, TrainingArguments, AutoTokenizer, set_seed
import torch

# Encoding
def encode(list_of_strings, pad_token_id=0):
    max_length = max([len(string) for string in list_of_strings])

    # create emtpy tensors
    attention_masks = torch.zeros((len(list_of_strings), max_length), dtype=torch.long)
    input_ids = torch.full((len(list_of_strings), max_length), pad_token_id, dtype=torch.long)

    for idx, string in enumerate(list_of_strings):
        # make sure string is in byte format
        if not isinstance(string, bytes):
            string = str.encode(string)

        input_ids[idx, :len(string)] = torch.tensor([x + 2 for x in string])
        attention_masks[idx, :len(string)] = 1

    return input_ids, attention_masks
    
# Decoding
def decode(outputs_ids):
    decoded_outputs = []
    for output_ids in outputs_ids.tolist():
        # transform id back to char IDs < 2 are simply transformed to ""
        decoded_outputs.append("".join([chr(x - 2) if x > 1 else "" for x in output_ids]))
    return decoded_outputs

from transformers import ReformerModelWithLMHead

model = ReformerModelWithLMHead.from_pretrained("google/reformer-enwik8")
encoded, attention_masks = encode(["In 1965, Brooks left IBM to found the Department of"])
decode(model.generate(encoded, do_sample=True, max_length=150))

# Train

1. 看微信推送增加规则，比如增加一个sent2，从之前的error找到最典型的无法处理的情况，比如looooooooove，合成sent2送给model
2. 查看hs2s以及monoise用我们所有metrics的结果，看看是不是可比的
3. 完成tweet-copy任务，可以作为Intermediate fine-tuning任务
3. 每次eval输出20个例子看看结果
----------------------------------------------
1. 训练token cls任务.假设3能全对训练增加align-embed过后的model.如果3和4结果都合理，考虑3、4结合到一起的流程.或者改造成类似bert预训练一样的两个loss结合的任务
2. figure out为什么bert2bert>>others
3. 在翻译后的结果和翻译之前跑某个sentiment analysis任务
4. 关于新metrics
5. 预约周五讨论和下周二讨论
---------------------------------------------
multi-task and Intermediate fine-tuning with token cls


In [1]:
import nlp
import logging
from datasets import load_metric
from transformers import EncoderDecoderModel, Trainer, TrainingArguments, AutoTokenizer, set_seed
from datasets import Dataset
import pandas as pd
from tweetNormalizer import normalizeTweet
from nltk.tokenize import TweetTokenizer
from emoji import demojize
from collections import defaultdict
import numpy as np

nltk_tokenizer = TweetTokenizer()

logging.basicConfig(level=logging.INFO)

SEED = 42
SAME_INOUTPUT_RATE = 0
TWEET_COPY_TASK = 0

#change 6 places

#bert-base-uncased, gpt2, roberta-base, vinai/bertweet-base, google/electra-base-discriminator(only encoder)
RUN_NAME="bert2bert_notebook"
ENCODER = "bert-base-uncased"
DECODER = "bert-base-uncased"
tie_ENCODER_DECODER=True
OUTPUT_DIR="./models/"+RUN_NAME+"/2/"

RUN_NAME="bert2bert_notebook"
RUN_NAME = "zonghaiyao tweetnorm " + RUN_NAME
ENTITY, PROJECT, NAME = RUN_NAME.split()

batch_size = 16   # set batch size here
encoder_length = 128
decoder_length = 128

PATH_TO_TRAIN_DATA = "WNUT2015_dataset/train_data.json"
PATH_TO_VAL_DATA = "WNUT2015_dataset/test_truth.json"
is_alignEmbed = True

rouge = load_metric('rouge', experiment_id=9)
bleu = load_metric('bleu', experiment_id=9)

#---------------------------------------------------------------------------------------------
# encoder tokenizer
encoder_tokenizer = AutoTokenizer.from_pretrained(ENCODER)

if ENCODER=="bert-base-uncased" or "google/electra-base-discriminator":
    # CLS token will work as BOS token, SEP token will work as EOS token
    encoder_tokenizer.bos_token = encoder_tokenizer.cls_token
    encoder_tokenizer.eos_token = encoder_tokenizer.sep_token
    
# decoder tokenizer
if DECODER=="gpt2":
    # make sure GPT2 appends EOS in begin and end
    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
        return outputs
    
    AutoTokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens
    decoder_tokenizer = AutoTokenizer.from_pretrained(DECODER)
    # set pad_token_id to unk_token_id -> be careful here as unk_token_id == eos_token_id == bos_token_id
    decoder_tokenizer.pad_token = decoder_tokenizer.unk_token
else:   
    decoder_tokenizer = AutoTokenizer.from_pretrained(DECODER)

if DECODER=="bert-base-uncased":
    # CLS token will work as BOS token, SEP token will work as EOS token
    decoder_tokenizer.bos_token = decoder_tokenizer.cls_token
    decoder_tokenizer.eos_token = decoder_tokenizer.sep_token
    
# set EncoderDecoderModel
model = EncoderDecoderModel.from_encoder_decoder_pretrained(ENCODER, DECODER, tie_encoder_decoder = tie_ENCODER_DECODER)

# set decoding params
model.config.decoder_start_token_id = decoder_tokenizer.bos_token_id
model.config.eos_token_id = decoder_tokenizer.eos_token_id
model.config.max_length = decoder_length
model.config.min_length = 0
model.config.no_repeat_ngram_size = 3
model.early_stopping = True
model.length_penalty = 2.0
model.num_beams = 4

INFO:filelock:Lock 46917127741328 acquired on /home/zonghaiyao/.cache/huggingface/datasets/dcd4134ec0ad23f318793c6f8b77745d97efebf4b194bcb3c1ce90f867bec0cc.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
INFO:filelock:Lock 46917127741328 released on /home/zonghaiyao/.cache/huggingface/datasets/dcd4134ec0ad23f318793c6f8b77745d97efebf4b194bcb3c1ce90f867bec0cc.a89f1fa0750909f2d149b1ecabc808fb66cb865c94bb8bbb135c55deb50da2d7.py.lock
INFO:filelock:Lock 46917127905040 acquired on /home/zonghaiyao/.cache/huggingface/datasets/c7db30bf448719bd2c2ee7c233832963ab2e0b85e984dda4f577016390fa0e85.7927df63b30f94ac549ad2d2e3c61c5089402aacb0ab0478007e0abfe3431378.py.lock
INFO:filelock:Lock 46917127905040 released on /home/zonghaiyao/.cache/huggingface/datasets/c7db30bf448719bd2c2ee7c233832963ab2e0b85e984dda4f577016390fa0e85.7927df63b30f94ac549ad2d2e3c61c5089402aacb0ab0478007e0abfe3431378.py.lock
Some weights of BertModel were not initialized from the model checkpoint at bert-bas

In [2]:
#-------------------------------------------------------------------------------------------
# load train and validation data
train_dataset = pd.read_json(PATH_TO_TRAIN_DATA, orient="records")
val_dataset = pd.read_json(PATH_TO_VAL_DATA, orient="records")

#---------------------------------
#do some normalization ourselves
def norm(token):
    if token.lower().startswith("@"):
        return "username"
    elif token.lower().startswith('#'):
        return "hashtag"
    elif token.lower().startswith("http") or token.lower().startswith("www"):
        return "httpurl"
    else:
        return token.replace("’", "'").replace("…", "...")

def pre_pocessing_input(x):
    result = []
    for item in x:
        item = norm(item)
        #the reason for encode+decode is "?!!" need to be "? ! !"
        input_ids = encoder_tokenizer(item).input_ids
        item = encoder_tokenizer.decode(input_ids, skip_special_tokens=True)
        result.append(item)
    return result

def pre_pocessing_output(x):
    result = []
    for item in x:
        item = norm(item)
        #the reason for encode+decode is "?!!" need to be "? ! !"
        input_ids = decoder_tokenizer(item).input_ids
        item = decoder_tokenizer.decode(input_ids, skip_special_tokens=True)
        result.append(item)
    return result

train_dataset['input'] = train_dataset['input'].apply(pre_pocessing_input)
train_dataset['output'] = train_dataset['output'].apply(pre_pocessing_output)
val_dataset['input'] = val_dataset['input'].apply(pre_pocessing_input)
val_dataset['output'] = val_dataset['output'].apply(pre_pocessing_output)
#-----------------------------------------
#check if it is tweet-copy task
if TWEET_COPY_TASK == 1:
    make_input_output_same = lambda x: x['output'].copy()
    train_dataset['input'] = train_dataset.apply(make_input_output_same, axis=1)
    val_dataset['input'] = val_dataset.apply(make_input_output_same, axis=1)
#do some data augumentation
elif SAME_INOUTPUT_RATE > 0: 
    train_dataset_no_chage_data = train_dataset.copy()
    train_dataset_no_chage_data = train_dataset_no_chage_data.sample(frac=1, random_state=SEED)
    origin_input_size = train_dataset_no_chage_data.shape[0]
    train_dataset_no_chage_data = train_dataset_no_chage_data.iloc[:int(SAME_INOUTPUT_RATE * origin_input_size)]
    make_input_output_same = lambda x: x['output'].copy()
    train_dataset_no_chage_data['input'] = train_dataset_no_chage_data.apply(make_input_output_same, axis=1)
    train_dataset = train_dataset.append(train_dataset_no_chage_data, ignore_index=True)
    train_dataset = train_dataset.sample(frac=1, random_state=SEED).reset_index(drop=True)

#-------------------------------------------
#make sentence for token list
make_sentence = lambda x : " ".join(x).lower()

train_dataset['input_sentence'] = train_dataset['input'].apply(make_sentence)
train_dataset['output_sentence'] = train_dataset['output'].apply(make_sentence)
val_dataset['input_sentence'] = val_dataset['input'].apply(make_sentence)
val_dataset['output_sentence'] = val_dataset['output'].apply(make_sentence)

#------------------------------------------
#make is_labels_need_change for previous metrics
#also calculate token_align_ids here, only for input, so use encode_tokenizer
labels_word2id = {}
def make_special_input_output(x):
    assert len(x['input']) == len(x['output'])
    
    input_token = []
    output_token = []
    
    for i in range(len(x['input'])):
        input_token.append(x['input'][i].split())
        output_token.append(x['output'][i].split())

    #then we make is_labels_need_change
    result = {}
    is_labels_need_change = []
    #align_ids for bos token is 0
    align_ids = [0]
    for i in range(len(input_token)):
        if len(input_token[i]) == len(output_token[i]):
            for j in range(len(output_token[i])):
                #add token into labels_word2id dict
                if output_token[i][j] not in labels_word2id.keys():
                    labels_word2id[output_token[i][j]] = len(labels_word2id)
                #they are the same, no need change, and align_ids + 0(keep) * ids_num
                if output_token[i][j].lower() == input_token[i][j].lower():
                    is_labels_need_change.append([labels_word2id[output_token[i][j]], 0])
                    length = len(encoder_tokenizer(input_token[i][j].lower(), add_special_tokens=False).input_ids)
                    align_ids.extend(list(np.zeros(length, dtype = np.int8)))
                #they are diff, need change, and align_ids + 1(norm) * ids_num
                else:
                    is_labels_need_change.append([labels_word2id[output_token[i][j]], 1])
                    length = len(encoder_tokenizer(input_token[i][j].lower(), add_special_tokens=False).input_ids)
                    align_ids.extend(list(np.ones(length, dtype = np.int8)))
        else:
            for j in range(len(output_token[i])):
                #add token into labels_word2id dict
                if output_token[i][j] not in labels_word2id.keys():
                    labels_word2id[output_token[i][j]] = len(labels_word2id)
                #they are diff, need change
                is_labels_need_change.append([labels_word2id[output_token[i][j]], 1])
            for j in range(len(input_token[i])):
                #they are diff, align_ids + 1(norm) * ids_num
                length = len(encoder_tokenizer(input_token[i][j].lower(), add_special_tokens=False).input_ids)
                align_ids.extend(list(np.ones(length, dtype = np.int8)))
    
    #align_ids for eos token is 0
    align_ids.append(0)
    #pad 0 to max_encoder_length
    while len(align_ids) < encoder_length : align_ids.append(0)
    
    return is_labels_need_change, align_ids

train_dataset[['is_labels_need_change', 'align_ids']] = train_dataset.apply(make_special_input_output, axis=1, result_type="expand")
val_dataset[['is_labels_need_change', 'align_ids']] = val_dataset.apply(make_special_input_output, axis=1, result_type="expand")


train_dataset = Dataset.from_pandas(train_dataset)
val_dataset = Dataset.from_pandas(val_dataset)

labels_id2word = {v: k for k, v in labels_word2id.items()}
#-------------------------------------------------------

In [3]:
# # for check the correctness of pre-pocessing
# i = 9

# print(train_dataset.iloc[i]['output'])
# print(train_dataset.iloc[i]['input'])
# print(train_dataset.iloc[i]['input_sentence'])
# print(train_dataset.iloc[i]['output_sentence'])
# print(train_dataset.iloc[i]["align_ids"])
# print(train_dataset.iloc[i]["is_labels_need_change"])

['rt', 'username', ':', 'words', 'that', 'describe', 'exo', ':', 'luhan', 'is', 'pain', 'chanyeol', 'is', 'sex', 'sehun', 'is', 'porn']
['rt', 'username', ':', 'words', 'that', 'describe', 'exo', ':', 'luhan', 'is', 'pain', 'chanyeol', 'is', 'sex', 'sehun', 'is', 'porn']
rt username : words that describe exo : luhan is pain chanyeol is sex sehun is porn
rt username : words that describe exo : luhan is pain chanyeol is sex sehun is porn
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[0, 0], [1, 0], [2, 0], [90, 0], [88, 0], [91, 0], [92, 0], [2, 0], [93, 0], [94, 0], [95, 0], [96, 0], [94, 0], [97, 0], [98, 0], [94, 0], [99, 0]]


In [5]:
global batch_copy
batch_copy = None

# map data correctly
def map_to_encoder_decoder_inputs(batch):
    global batch_copy
    batch_copy = batch
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = encoder_tokenizer(batch["input_sentence"], padding="max_length", truncation=True, max_length=encoder_length)
    outputs = decoder_tokenizer(batch["output_sentence"], padding="max_length", truncation=True, max_length=decoder_length)

    batch["input_ids"] = inputs.input_ids
    if is_alignEmbed:
        batch["token_align_ids"] = batch["align_ids"]
   
    batch["is_labels_need_change"] = batch["is_labels_need_change"]
    batch["attention_mask"] = inputs.attention_mask

    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask
    
    
    if DECODER=="gpt2":
        # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
        batch["labels"] = [
            [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
        ]
    else:
        # mask loss for padding
        batch["labels"] = [
            [-100 if token == decoder_tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]
        ]
    

    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])
    
    batch_copy = batch
    
    return batch

In [6]:
# make train dataset ready
train_dataset = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["input_sentence", "output_sentence"],
)

if is_alignEmbed:
    train_dataset.set_format(
        type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels", "is_labels_need_change", "token_align_ids"],
    )
else:
    train_dataset.set_format(
        type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels", "is_labels_need_change"],
    )

# same for validation dataset
val_dataset = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=batch_size, remove_columns=["input_sentence", "output_sentence"],
)

if is_alignEmbed:
    val_dataset.set_format(
        type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels", "is_labels_need_change", "token_align_ids"],
    )
else:
    val_dataset.set_format(
        type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels", "is_labels_need_change"],
    )

In [7]:
#-----------------------------------------------------------------------------------
# global record
# record = {}
# load metrics for validation
# Calculate previous_metrics score of a corpus.
def previous_metrics(pred):
#     global record
    pred_ids = pred.predictions
    is_labels_need_change = pred.is_labels_need_change

    def batch_convert_ids_to_tokens(sequences, **kwargs):
        return [decoder_tokenizer.decode(seq, **kwargs) for seq in sequences]

    pred_tokens = batch_convert_ids_to_tokens(pred_ids, skip_special_tokens=True)

    assert len(pred_tokens) == len(is_labels_need_change)
    norm_correct, norm_wrong, keep_correct, keep_wrong = 0.0, 0.0, 0.0, 0.0

    for pred_tokens, oracle_tokens in zip(pred_tokens, is_labels_need_change):
        pred_tokens = pred_tokens.split()
        sent_length = len(oracle_tokens)

        while len(pred_tokens) < len(oracle_tokens) : pred_tokens.append("<PAD>")
#         print("pred_tokens:", pred_tokens)
#         print("oracle_tokens:", oracle_tokens)
#         record['pred_tokens'] = pred_tokens
#         record['oracle_tokens'] = oracle_tokens
        
        for i in range(sent_length):
            pred_token = pred_tokens[i]
            oracle_token = labels_id2word[int(oracle_tokens[i][0])]
            token_need_change = oracle_tokens[i][1]
            #norm-correct：需要改且改对的
            if token_need_change == 1 and oracle_token.lower() == pred_token.lower() and oracle_token.strip():
                norm_correct += 1
            #norm-wrong ：需要改但没改对的
            if token_need_change == 1 and oracle_token.lower() != pred_token.lower() and oracle_token.strip():
                norm_wrong += 1
            #keep-correct：不需要改且没有改的
            if token_need_change == 0 and oracle_token.lower() == pred_token.lower() and oracle_token.strip():
                keep_correct += 1
            #keep-wrong ：不需要改但是改了的
            if token_need_change == 0 and oracle_token.lower() != pred_token.lower() and oracle_token.strip():
                keep_wrong += 1
    
    #所有不需要修改token的正确率
    correct_of_all_need_keep = keep_correct / (keep_correct + keep_wrong)

    results = {}
    results["keep_correct"] = keep_correct
    results["keep_wrong"] = keep_wrong
    results["correct_of_all_need_keep"] = correct_of_all_need_keep
    
    if TWEET_COPY_TASK == 0:
        #所有需要修改token的正确率
        correct_of_all_need_norm = norm_correct / (norm_correct + norm_wrong)
        #输出正确token的正确率
        correct_token = (norm_correct + keep_correct) / (norm_correct + norm_wrong + keep_correct + keep_wrong)
        results["norm_correct"] = norm_correct
        results["norm_wrong"] = norm_wrong
        results["correct_of_all_need_norm"] = correct_of_all_need_norm
        results["correct_token"] = correct_token
    
    return results

def compute_metrics(pred):
    #Calculate previous_metrics score of a corpus
    previous_metrics_results = previous_metrics(pred)
    
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = decoder_tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = decoder_tokenizer.eos_token_id
    label_str = decoder_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    metrics_rouge = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1", "rouge2", "rouge3", "rouge4", "rougeL", "rougeLsum"])

    def batch_convert_ids_to_tokens(sequences, **kwargs):
        return [decoder_tokenizer.convert_ids_to_tokens(seq, **kwargs) for seq in sequences]
    
    pred_tokens = batch_convert_ids_to_tokens(pred_ids, skip_special_tokens=True)
    
    def batch_convert_ids_to_tokens(sequences, **kwargs):
        return [[decoder_tokenizer.convert_ids_to_tokens(seq, **kwargs)] for seq in sequences]
    
    label_tokens = batch_convert_ids_to_tokens(labels_ids, skip_special_tokens=True)
    
    metrics_bleu = bleu.compute(predictions=pred_tokens, references=label_tokens)
    
    if TWEET_COPY_TASK == 1:
        return {
            "rouge1_precision": round(metrics_rouge['rouge1'].mid.precision, 4),
            "rouge1_recall": round(metrics_rouge['rouge1'].mid.recall, 4),
            "rouge1_fmeasure": round(metrics_rouge['rouge1'].mid.fmeasure, 4),
            "rouge2_precision": round(metrics_rouge['rouge2'].mid.precision, 4),
            "rouge2_recall": round(metrics_rouge['rouge2'].mid.recall, 4),
            "rouge2_fmeasure": round(metrics_rouge['rouge2'].mid.fmeasure, 4),
            "rouge3_precision": round(metrics_rouge['rouge3'].mid.precision, 4),
            "rouge3_recall": round(metrics_rouge['rouge3'].mid.recall, 4),
            "rouge3_fmeasure": round(metrics_rouge['rouge3'].mid.fmeasure, 4),
            "rouge4_precision": round(metrics_rouge['rouge4'].mid.precision, 4),
            "rouge4_recall": round(metrics_rouge['rouge4'].mid.recall, 4),
            "rouge4_fmeasure": round(metrics_rouge['rouge4'].mid.fmeasure, 4),
            "rougeL_precision": round(metrics_rouge['rougeL'].mid.precision, 4),
            "rougeL_recall": round(metrics_rouge['rougeL'].mid.recall, 4),
            "rougeL_fmeasure": round(metrics_rouge['rougeL'].mid.fmeasure, 4),
            "rougeLsum_precision": round(metrics_rouge['rougeLsum'].mid.precision, 4),
            "rougeLsum_recall": round(metrics_rouge['rougeLsum'].mid.recall, 4),
            "rougeLsum_fmeasure": round(metrics_rouge['rougeLsum'].mid.fmeasure, 4),
            "bleu": round(metrics_bleu['bleu'], 4),
            "keep_correct": round(previous_metrics_results['keep_correct'], 4),
            "keep_wrong": round(previous_metrics_results['keep_wrong'], 4),
            "correct_of_all_need_keep": round(previous_metrics_results['correct_of_all_need_keep'], 4),
        }
    else:
        return {
            "rouge1_precision": round(metrics_rouge['rouge1'].mid.precision, 4),
            "rouge1_recall": round(metrics_rouge['rouge1'].mid.recall, 4),
            "rouge1_fmeasure": round(metrics_rouge['rouge1'].mid.fmeasure, 4),
            "rouge2_precision": round(metrics_rouge['rouge2'].mid.precision, 4),
            "rouge2_recall": round(metrics_rouge['rouge2'].mid.recall, 4),
            "rouge2_fmeasure": round(metrics_rouge['rouge2'].mid.fmeasure, 4),
            "rouge3_precision": round(metrics_rouge['rouge3'].mid.precision, 4),
            "rouge3_recall": round(metrics_rouge['rouge3'].mid.recall, 4),
            "rouge3_fmeasure": round(metrics_rouge['rouge3'].mid.fmeasure, 4),
            "rouge4_precision": round(metrics_rouge['rouge4'].mid.precision, 4),
            "rouge4_recall": round(metrics_rouge['rouge4'].mid.recall, 4),
            "rouge4_fmeasure": round(metrics_rouge['rouge4'].mid.fmeasure, 4),
            "rougeL_precision": round(metrics_rouge['rougeL'].mid.precision, 4),
            "rougeL_recall": round(metrics_rouge['rougeL'].mid.recall, 4),
            "rougeL_fmeasure": round(metrics_rouge['rougeL'].mid.fmeasure, 4),
            "rougeLsum_precision": round(metrics_rouge['rougeLsum'].mid.precision, 4),
            "rougeLsum_recall": round(metrics_rouge['rougeLsum'].mid.recall, 4),
            "rougeLsum_fmeasure": round(metrics_rouge['rougeLsum'].mid.fmeasure, 4),
            "bleu": round(metrics_bleu['bleu'], 4),
            "norm_correct": round(previous_metrics_results['norm_correct'], 4),
            "norm_wrong": round(previous_metrics_results['norm_wrong'], 4),
            "keep_correct": round(previous_metrics_results['keep_correct'], 4),
            "keep_wrong": round(previous_metrics_results['keep_wrong'], 4),
            "correct_of_all_need_norm": round(previous_metrics_results['correct_of_all_need_norm'], 4),
            "correct_of_all_need_keep": round(previous_metrics_results['correct_of_all_need_keep'], 4),
            "correct_token": round(previous_metrics_results['correct_token'], 4),
        }
        
#----------------------------------------------------------------------------------------
# begin train

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_from_generate=True,
    evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    logging_steps=20,
    save_steps=100,
    eval_steps=20,
    overwrite_output_dir=True,
    warmup_steps=50,
    save_total_limit=3,
    num_train_epochs=30,

    
    seed=SEED,
    fp16=True,
    run_name=RUN_NAME,
)


# instantiate trainer
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    my_trainer = True
)


# start training
trainer.train()

/mnt/nfs/work1/llcao/zonghaiyao/tweetNorm/datasets/src/datasets/arrow_dataset.py:835: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629403081/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


/home/zonghaiyao/anaconda3/envs/tweetNorm/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


ERROR:wandb.jupyter:Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: iesl-boxes (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.6 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20201015_173118-2mmfk6ks
wandb: Syncing run bert2bert_notebook



{'loss': 9.056087493896484, 'learning_rate': 2e-05, 'epoch': 20.0, 'total_flos': 42538989772800, 'step': 20}


/mnt/nfs/work1/llcao/zonghaiyao/tweetNorm/transformers/src/transformers/trainer.py:1449: FutureWarning: The `_prediction_loop` method is deprecated and won't be called in a future version, define `prediction_loop` in your subclass.
  FutureWarning,


Setting `pad_token_id` to 102 (first `eos_token_id`) to generate sequence
INFO:filelock:Lock 46917422570384 acquired on /home/zonghaiyao/.cache/huggingface/metrics/rouge/default/9-1-0.arrow.lock
INFO:filelock:Lock 46917422570384 released on /home/zonghaiyao/.cache/huggingface/metrics/rouge/default/9-1-0.arrow.lock
INFO:filelock:Lock 46917422513744 acquired on /home/zonghaiyao/.cache/huggingface/metrics/rouge/default/9-1-0.arrow.lock


INFO:/mnt/nfs/work1/llcao/zonghaiyao/tweetNorm/datasets/src/datasets/metric.py:Removing /home/zonghaiyao/.cache/huggingface/metrics/rouge/default/9-1-0.arrow
INFO:filelock:Lock 46917422513744 released on /home/zonghaiyao/.cache/huggingface/metrics/rouge/default/9-1-0.arrow.lock
INFO:filelock:Lock 46917422624848 acquired on /home/zonghaiyao/.cache/huggingface/metrics/bleu/default/9-1-0.arrow.lock
INFO:filelock:Lock 46917422624848 released on /home/zonghaiyao/.cache/huggingface/metrics/bleu/default/9-1-0.arrow.lock
INFO:filelock:Lock 46917426162960 acquired on /home/zonghaiyao/.cache/huggingface/metrics/bleu/default/9-1-0.arrow.lock
INFO:/mnt/nfs/work1/llcao/zonghaiyao/tweetNorm/datasets/src/datasets/metric.py:Removing /home/zonghaiyao/.cache/huggingface/metrics/bleu/default/9-1-0.arrow
INFO:filelock:Lock 46917426162960 released on /home/zonghaiyao/.cache/huggingface/metrics/bleu/default/9-1-0.arrow.lock


{'eval_loss': 7.7961297035217285, 'eval_rouge1_precision': 0.0332, 'eval_rouge1_recall': 0.085, 'eval_rouge1_fmeasure': 0.0473, 'eval_rouge2_precision': 0.0, 'eval_rouge2_recall': 0.0, 'eval_rouge2_fmeasure': 0.0, 'eval_rouge3_precision': 0.0, 'eval_rouge3_recall': 0.0, 'eval_rouge3_fmeasure': 0.0, 'eval_rouge4_precision': 0.0, 'eval_rouge4_recall': 0.0, 'eval_rouge4_fmeasure': 0.0, 'eval_rougeL_precision': 0.0301, 'eval_rougeL_recall': 0.0753, 'eval_rougeL_fmeasure': 0.0428, 'eval_rougeLsum_precision': 0.03, 'eval_rougeLsum_recall': 0.0759, 'eval_rougeLsum_fmeasure': 0.0428, 'eval_bleu': 0.0, 'eval_norm_correct': 0.0, 'eval_norm_wrong': 29.0, 'eval_keep_correct': 2.0, 'eval_keep_wrong': 151.0, 'eval_correct_of_all_need_norm': 0.0, 'eval_correct_of_all_need_keep': 0.0131, 'eval_correct_token': 0.011, 'epoch': 20.0, 'total_flos': 42538989772800, 'step': 20}



TrainOutput(global_step=30, training_loss=8.082954915364583)

In [2]:
# # Calculate previous_metrics score of a corpus.
# def previous_metrics(pred):
#     pred_ids = pred.predictions
#     is_labels_need_change = pred.is_labels_need_change

#     def batch_convert_ids_to_tokens(sequences, **kwargs):
#         return [decoder_tokenizer.decode(seq, **kwargs) for seq in sequences]

#     pred_tokens = batch_convert_ids_to_tokens(pred_ids, skip_special_tokens=True)

#     assert len(pred_tokens) == len(is_labels_need_change)
#     norm_correct, norm_wrong, keep_correct, keep_wrong = 0.0, 0.0, 0.0, 0.0

#     for pred_tokens, oracle_tokens in zip(pred_tokens, is_labels_need_change):
#         sent_length = len(oracle_tokens)
#         while len(pred_tokens) < len(oracle_tokens) : pred_tokens.append("<PAD>")
#         for i in range(sent_length):
#             pred_token = pred_tokens[i]
#             oracle_token = labels_id2word[int(oracle_tokens[i][0])]
#             token_need_change = oracle_tokens[i][1]
#             #norm-correct：需要改且改对的
#             if token_need_change == 1 and oracle_token.lower() == pred_token.lower() and oracle_token.strip():
#                 norm_correct += 1
#             #norm-wrong ：需要改但没改对的
#             if token_need_change == 1 and oracle_token.lower() != pred_token.lower() and oracle_token.strip():
#                 norm_wrong += 1
#             #keep-correct：不需要改且没有改的
#             if token_need_change == 0 and oracle_token.lower() == pred_token.lower() and oracle_token.strip():
#                 keep_correct += 1
#             #keep-wrong ：不需要改但是改了的
#             if token_need_change == 0 and oracle_token.lower() != pred_token.lower() and oracle_token.strip():
#                 keep_wrong += 1
#     #所有需要修改token的正确率
#     correct_of_all_need_norm = norm_correct / (norm_correct + norm_wrong)
#     #所有不需要修改token的正确率
#     correct_of_all_need_keep = keep_correct / (keep_correct + keep_wrong)
#     #输出正确token的正确率
#     correct_token = (norm_correct + keep_correct) / (norm_correct + norm_wrong + keep_correct + keep_wrong)

#     results = {}
#     results["norm_correct"] = norm_correct
#     results["norm_wrong"] = norm_wrong
#     results["keep_correct"] = keep_correct
#     results["keep_wrong"] = keep_wrong
#     results["correct_of_all_need_norm"] = correct_of_all_need_norm
#     results["correct_of_all_need_keep"] = correct_of_all_need_keep
#     results["correct_token"] = correct_token
#     return results

In [67]:
# previous_metrics(pred_1)

{'norm_correct': 0.0,
 'norm_wrong': 107.0,
 'keep_correct': 20.0,
 'keep_wrong': 1471.0,
 'correct_of_all_need_norm': 0.0,
 'correct_of_all_need_keep': 0.01341381623071764,
 'correct_token': 0.012515644555694618}

In [ ]:
# # load train and validation data
# train_df = pd.read_json("WNUT2015_dataset/train_data.json", orient="records")
# val_df = pd.read_json("WNUT2015_dataset/test_truth.json", orient="records")

In [12]:
# get_len = lambda x : len(x)

# train_df['input_length'] = train_df['input'].apply(get_len)
# train_df['output_length'] = train_df['output'].apply(get_len)
# val_df['input_length'] = val_df['input'].apply(get_len)
# val_df['output_length'] = val_df['output'].apply(get_len)

# print(max(train_df['input_length']), max(train_df['output_length']), max(val_df['input_length']), max(val_df['output_length']))
# print(min(train_df['input_length']), min(train_df['output_length']), min(val_df['input_length']), min(val_df['output_length']))


41 41 41 41
4 4 4 4
